In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy.stats import ttest_ind, skew

### Homework 2.1

In [2]:
hw21 = pd.read_csv('homework_2.1.csv', index_col=0)
hw21.head()

,time,G1,G2,G3
0,0,0.882026,1.441575,0.065409
1,1,0.210079,-0.163880,0.140310
2,2,0.509369,-0.115242,0.819830
3,3,1.150447,1.014698,0.607632
4,4,0.973779,-0.046562,0.610066


In [3]:
g1 = hw21['G1']
g2 = hw21['G2']
g3 = hw21['G3']
x = hw21['time']

In [4]:
model1 = sm.OLS(g1, sm.add_constant(x)).fit()
model1.params

const    0.104236
time     0.008498
dtype: float64

In [5]:
model2 = sm.OLS(g2, sm.add_constant(x)).fit()
model2.params

const    0.685523
time     0.007080
dtype: float64

In [6]:
model3 = sm.OLS(g3, sm.add_constant(x)).fit()
model3.params

const    0.147479
time     0.011473
dtype: float64

In [7]:
model1_mean = sm.OLS((g1 - np.mean(g1)), ((x - np.mean(x)))).fit()
model1_mean.params

time    0.008498
dtype: float64

In [8]:
model2_mean = sm.OLS((g2 - np.mean(g2)), ((x - np.mean(x)))).fit()
model2_mean.params

time    0.00708
dtype: float64

In [9]:
model3_mean = sm.OLS((g3 - np.mean(g3)), ((x - np.mean(x)))).fit()
model3_mean.params

time    0.011473
dtype: float64

In [10]:
(0.008498 + 0.007080 + 0.011473) / 3

0.009017

### Homework 2.2

In [11]:
hw22 = pd.read_csv('homework_2.2.csv', index_col=0)
hw22.head()

,X,Y,Z
0,0,1.182435,-0.725820
1,0,2.714474,0.563476
2,0,0.077612,-0.435632
3,0,-0.154449,-0.104553
4,0,22.298992,-2.321273


In [12]:
y_treat = hw22[hw22['X']==1]['Y']
y_untreat = hw22[hw22['X'] == 0]['Y']

In [13]:
mu_y_treat = y_treat.mean()
mu_y_untreat = y_untreat.mean()

In [14]:
mean_effect = mu_y_treat - mu_y_untreat
mean_effect

np.float64(2.92071726472051)

In [15]:
# Bootstrap sampling to measure variance of effect

def bootstrap_sample(data, n=None):
    if n is None:
        n = len(data)
    return data.sample(n, replace=True)

effect = []
for i in range(len(hw22)):
    bs = bootstrap_sample(hw22)
    y_treat = bs[bs['X'] == 1]['Y']
    y_untreat = bs[bs['X'] == 0]['Y']
    mu_y_treat = y_treat.mean()
    mu_y_untreat = y_untreat.mean()
    effect.append(mu_y_treat - mu_y_untreat)

np.var(effect)

np.float64(0.03225722404867962)

In [16]:
# Linear regression to measure effect

model = sm.OLS(hw22['Y'], sm.add_constant(hw22['X'])).fit()
model.params # coefficient of X is the effect

const    4.922040
X        2.920717
dtype: float64

In [21]:
bootstrap_effects = []

for i in range(len(hw22)):
    bs = bootstrap_sample(hw22)
    model_boot = sm.OLS(bs['Y'], sm.add_constant(bs['X'])).fit()
    bootstrap_effects.append(model_boot.params['X'])

effect_skew = skew(bootstrap_effects)

In [22]:
effect_skew

np.float64(0.0469345498826032)